In [1]:
import os
import pandas as pd
import numpy as np
from datetime import datetime, date
from dateutil.relativedelta import relativedelta
import math
import psycopg2
from sqlalchemy import create_engine
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler # conda install -c conda-forge imbalanced-learn

df_customer = pd.read_csv("./data/customer.csv")
df_category = pd.read_csv("./data/category.csv")
df_product = pd.read_csv("./data/product.csv")
df_receipt = pd.read_csv("./data/receipt.csv")
df_store = pd.read_csv("./data/store.csv")
df_geocode = pd.read_csv("./data/geocode.csv")

C:\Users\335395\AppData\Local\Continuum\anaconda3\envs\Yugo environment\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


P-056: 顧客データフレーム（df_customer）の年齢（age）をもとに10歳刻みで年代を算出し、顧客ID（customer_id）、生年月日（birth_day）とともに抽出せよ。ただし、60歳以上は全て60歳代とすること。年代を表すカテゴリ名は任意とする。先頭10件を表示させればよい。

In [2]:
df_customer.head()

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
0,CS021313000114,大野 あや子,1,女性,1981-04-29,37,259-1113,神奈川県伊勢原市粟窪**********,S14021,20150905,0-00000000-0
1,CS037613000071,六角 雅彦,9,不明,1952-04-01,66,136-0076,東京都江東区南砂**********,S13037,20150414,0-00000000-0
2,CS031415000172,宇多田 貴美子,1,女性,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C
3,CS028811000001,堀井 かおり,1,女性,1933-03-27,86,245-0016,神奈川県横浜市泉区和泉町**********,S14028,20160115,0-00000000-0
4,CS001215000145,田崎 美紀,1,女性,1995-03-29,24,144-0055,東京都大田区仲六郷**********,S13001,20170605,6-20090929-2


In [3]:
df1 = df_customer[['customer_id','birth_day','age']]
df1.head()

,customer_id,birth_day,age
0,CS021313000114,1981-04-29,37
1,CS037613000071,1952-04-01,66
2,CS031415000172,1976-10-04,42
3,CS028811000001,1933-03-27,86
4,CS001215000145,1995-03-29,24


In [4]:
df1['era'] = df1['age'].apply(lambda x: math.floor(x/10)*10)
df1.head(10)

C:\Users\335395\AppData\Local\Continuum\anaconda3\envs\Yugo environment\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,customer_id,birth_day,age,era
0,CS021313000114,1981-04-29,37,30
1,CS037613000071,1952-04-01,66,60
2,CS031415000172,1976-10-04,42,40
3,CS028811000001,1933-03-27,86,80
4,CS001215000145,1995-03-29,24,20
5,CS020401000016,1974-09-15,44,40
6,CS015414000103,1977-08-09,41,40
7,CS029403000008,1973-08-17,45,40
8,CS015804000004,1931-05-02,87,80
9,CS033513000180,1962-07-11,56,50


P-057: 前問題の抽出結果と性別（gender）を組み合わせ、新たに性別×年代の組み合わせを表すカテゴリデータを作成せよ。組み合わせを表すカテゴリの値は任意とする。先頭10件を表示させればよい。

In [10]:
df2 = pd.concat([df1,df_customer['gender']],axis=1)
df2.head()

,customer_id,birth_day,age,era,gender
0,CS021313000114,1981-04-29,37,30,女性
1,CS037613000071,1952-04-01,66,60,不明
2,CS031415000172,1976-10-04,42,40,女性
3,CS028811000001,1933-03-27,86,80,女性
4,CS001215000145,1995-03-29,24,20,女性


In [12]:
df2['era*sex'] = df2['era'].astype(str) + '代' + df2['gender']
df2.head(10)

,customer_id,birth_day,age,era,gender,era*sex
0,CS021313000114,1981-04-29,37,30,女性,30代女性
1,CS037613000071,1952-04-01,66,60,不明,60代不明
2,CS031415000172,1976-10-04,42,40,女性,40代女性
3,CS028811000001,1933-03-27,86,80,女性,80代女性
4,CS001215000145,1995-03-29,24,20,女性,20代女性
5,CS020401000016,1974-09-15,44,40,男性,40代男性
6,CS015414000103,1977-08-09,41,40,女性,40代女性
7,CS029403000008,1973-08-17,45,40,男性,40代男性
8,CS015804000004,1931-05-02,87,80,男性,80代男性
9,CS033513000180,1962-07-11,56,50,女性,50代女性


P-058: 顧客データフレーム（df_customer）の性別コード（gender_cd）をダミー変数化し、顧客ID（customer_id）とともに抽出せよ。結果は10件表示させれば良い。

In [19]:
df3 = df_customer[['customer_id','gender_cd']]
df3 = pd.get_dummies(df3,columns=['gender_cd'])
df3.head()

,customer_id,gender_cd_0,gender_cd_1,gender_cd_9
0,CS021313000114,0,1,0
1,CS037613000071,0,0,1
2,CS031415000172,0,1,0
3,CS028811000001,0,1,0
4,CS001215000145,0,1,0


P-059: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、合計した売上金額を平均0、標準偏差1に標準化して顧客ID、売上金額合計とともに表示せよ。標準化に使用する標準偏差は、不偏標準偏差と標本標準偏差のどちらでも良いものとする。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。結果は10件表示させれば良い。

In [21]:
df_receipt.head()

,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
0,20181103,1257206400,S14006,112,1,CS006214000001,P070305012,1,158
1,20181118,1258502400,S13008,1132,2,CS008415000097,P070701017,1,81
2,20170712,1215820800,S14028,1102,1,CS028414000014,P060101005,1,170
3,20190205,1265328000,S14042,1132,1,ZZ000000000000,P050301001,1,25
4,20180821,1250812800,S14025,1102,2,CS025415000050,P060102007,1,90


In [29]:
df_z = df_receipt.query('not customer_id.str.startswith("Z")',engine='python')
df4 = df_z.groupby('customer_id').agg({'amount':'sum'}).reset_index()
df4['amount_scale'] = preprocessing.scale(df4['amount'])
df4.head(10)

,customer_id,amount,amount_scale
0,CS001113000004,1298,-0.459378
1,CS001114000005,626,-0.706390
2,CS001115000010,3044,0.182413
3,CS001205000004,1988,-0.205749
4,CS001205000006,3337,0.290114
5,CS001211000025,456,-0.768879
6,CS001212000027,448,-0.771819
7,CS001212000031,296,-0.827691
8,CS001212000046,228,-0.852686
9,CS001212000070,456,-0.768879


P-060: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、合計した売上金額を最小値0、最大値1に正規化して顧客ID、売上金額合計とともに表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。結果は10件表示させれば良い。

In [30]:
df5 = df_z.groupby('customer_id').agg({'amount':'sum'}).reset_index()
df5['amount_scale'] = preprocessing.minmax_scale(df5['amount'])
df5.head(10)

,customer_id,amount,amount_scale
0,CS001113000004,1298,0.053354
1,CS001114000005,626,0.024157
2,CS001115000010,3044,0.129214
3,CS001205000004,1988,0.083333
4,CS001205000006,3337,0.141945
5,CS001211000025,456,0.016771
6,CS001212000027,448,0.016423
7,CS001212000031,296,0.009819
8,CS001212000046,228,0.006865
9,CS001212000070,456,0.016771
